In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
Data = pd.read_csv("../input/udemy-courses/udemy_courses.csv")



In [ ]:
Data.info()
Data[0:10]

# Distribution of is_paid : True or False

In [ ]:
cnt_pro = Data['is_paid'].value_counts()
plt.figure(figsize=(6,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of Data', fontsize=12)
plt.xlabel('is_paid', fontsize=12)
plt.xticks(rotation=80)
plt.show();

# Subject

In [ ]:
cnt_pro = Data['subject'].value_counts()
plt.figure(figsize=(6,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of subject', fontsize=12)
plt.xlabel('Subject', fontsize=12)
plt.xticks(rotation=80)
plt.show();

# Number of reviews

# All Levels

In [ ]:
num_reviews= Data[Data['level']=='All Levels'].groupby(['is_paid']).agg({'num_reviews':['sum']})
num_lectures = Data[Data['level']=='All Levels'].groupby(['is_paid']).agg({'num_lectures':['sum']})
total= num_reviews.join(num_lectures)


plt.figure(figsize=(15,10))
plt.subplot(2, 2, 1)
total.plot(ax=plt.gca(), title='All Levels')

# Beginner Level

In [ ]:
num_reviews= Data[Data['level']=='Beginner Level'].groupby(['is_paid']).agg({'num_reviews':['sum']})
num_lectures = Data[Data['level']=='Beginner Level'].groupby(['is_paid']).agg({'num_lectures':['sum']})
total= num_reviews.join(num_lectures)


plt.figure(figsize=(15,10))
plt.subplot(2, 2, 1)
total.plot(ax=plt.gca(), title='Beginner Level')

# Intermediate Level

In [ ]:
num_reviews= Data[Data['level']=='Intermediate Level'].groupby(['is_paid']).agg({'num_reviews':['sum']})
num_lectures = Data[Data['level']=='Intermediate Level'].groupby(['is_paid']).agg({'num_lectures':['sum']})
total= num_reviews.join(num_lectures)


plt.figure(figsize=(15,10))
plt.subplot(2, 2, 1)
total.plot(ax=plt.gca(), title='Intermediate Level')

# Expert Level

In [ ]:
num_reviews= Data[Data['level']=='Expert Level'].groupby(['is_paid']).agg({'num_reviews':['sum']})
num_lectures = Data[Data['level']=='Expert Level'].groupby(['is_paid']).agg({'num_lectures':['sum']})
total= num_reviews.join(num_lectures)


plt.figure(figsize=(15,10))
plt.subplot(2, 2, 1)
total.plot(ax=plt.gca(), title='Expert Level')

# Top 30 Most Popular Courses

In [ ]:
#Top 30 Most Popular Courses by num_subscribers
top_course = Data.sort_values(by='num_subscribers', ascending=False)[:30]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_course.course_title, x=top_course.num_subscribers)
plt.xticks()
plt.xlabel('num_subscribers')
plt.ylabel('Course_title')
plt.title('The Most Popular Courses')
plt.show()

# The 30  Most Popular Courses Reviews

In [ ]:
#Top 30 Most Popular Courses Reviews by num_reviews
top_course = Data.sort_values(by='num_reviews', ascending=False)[:30]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_course.course_title, x=top_course.num_reviews)
plt.xticks()
plt.xlabel('num_reviews')
plt.ylabel('Course_title')
plt.title('The Most Popular Courses Reviews')
plt.show()

# Top 30  Num_lectures

In [ ]:
#Top 30 Num_lectures by num_lectures
top_course = Data.sort_values(by='num_lectures', ascending=False)[:30]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_course.course_title, x=top_course.num_lectures)
plt.xticks()
plt.xlabel('num_lectures')
plt.ylabel('Course_title')
plt.title('Num_lectures Courses')
plt.show()

# Plotting Heatmap
Heatmap can be defined as a method of graphically representing numerical data where individual data points contained in the matrix are represented using different colors. The colors in the heatmap can denote the frequency of an event, the performance of various metrics in the data set, and so on. Different color schemes are selected by varying businesses to present the data they want to be plotted on a heatmap [2].

In [ ]:
Data1= Data[['is_paid','price','num_subscribers','num_reviews','num_lectures','content_duration','subject']] #Subsetting the data
cor = Data.corr() #Calculate the correlation of the above variables
sns.heatmap(cor, square = True) #Plot the correlation as heat map

As you can see above, we obtain the heatmap of correlation among the variables. The color palette in the side represents the amount of correlation among the variables. The lighter shade represents a high correlation.

# Prediction: Udemy Free Courses or Paid Courses

This work try to predict free courses or paid courses using Random Forest.

Variable prediction:
1. is_paid	price
1. num_subscribers	
1. num_reviews	
1. num_lectures	
1. content_duration	
1. subject

Target 
* is_paid : True , False



In [ ]:
Data1.head()

In [ ]:
#Frequency distribution of classes"
train_outcome = pd.crosstab(index=Data["is_paid"],  # Make a crosstab
                              columns="count")      # Name the count column

train_outcome

Transfer sting to numeric

In [ ]:
#Select feature column names and target variable we are going to use for training
subject = {'Web Development': 1 ,'Business Finance': 2, 'Musical Instruments': 3, 'Graphic Design': 4} 
Data1.subject = [subject[item] for item in Data1.subject] 
print(Data1)

In [ ]:
print("Any missing sample in test set:",Data1.isnull().values.any(), "\n")

# SPLITING DATA
Data for training and testing To select a set of training data that will be input in the Machine Learning algorithm, to ensure that the classification algorithm training can be generalized well to new data. For this study using a sample size of 20% ( aims to reduce the overfitting effect).

In [ ]:
from sklearn.model_selection import train_test_split
Y = Data1['is_paid']
X = Data1.drop(columns=['is_paid'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=9)

In [ ]:
print('X train shape: ', X_train.shape)
print('Y train shape: ', Y_train.shape)
print('X test shape: ', X_test.shape)
print('Y test shape: ', Y_test.shape)

In [ ]:
X_test

In [ ]:
Y_test

# Random forest
To understand the random forest model, we must first learn about the decision tree, the basic building block of a random forest. We need to talk about trees before we can get into forests.   A decision tree is a flowchart-like tree structure where an internal node represents feature, the branch represents a decision rule, and each leaf node represents the outcome. The decision tree analyzes a set of data to construct a set of rules or questions, which are used to predict a class, i.e., the goal of decision tree is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. In this sense the decision tree selects the best attribute using to divide the records, converting that attribute into a decision node and dividing the data set into smaller subsets, to finally start the construction of the tree repeating this process recursively.

Example
![https://cdn-images-1.medium.com/max/824/0*J2l5dvJ2jqRwGDfG.png](https://cdn-images-1.medium.com/max/824/0*J2l5dvJ2jqRwGDfG.png)


Principle of the decision tree’s algorithm

* Root Node: This attribute is used for dividing the data into two or more sets. The feature attribute in this node is selected based on Attribute Selection Techniques.
* Branch or Sub-Tree: A part of the entire decision tree is called branch or sub-tree.
* Splitting: Dividing a node into two or more sub-nodes based on if-else conditions.
* Decision Node: After splitting the sub-nodes into further sub-nodes, then it is called as the decision node.
* Leaf or Terminal Node: This is the end of the decision tree where it cannot be split into further sub-nodes.
* Pruning: Removing a sub-node from the tree is called pruning.


![https://cdn-images-1.medium.com/max/688/0*pb-1ufHK-OmR8k7r.png](https://cdn-images-1.medium.com/max/688/0*pb-1ufHK-OmR8k7r.png)



Source: https://www.kdnuggets.com/

# Working of Random forest classification
Random forest is a supervised learning algorithm that creates a forest randomly. This forest, is a set of decision trees, most of the times trained with the bagging method. The essential idea of bagging is to average many noisy but approximately impartial models, and therefore reduce the variation. Each tree is constructed using the following algorithm:

* Let $N$ be the number of test cases, $M$ is the number of variables in the classifier.
* Let $m$ be the number of input variables to be used to determine the decision in a given node; $m<M$.
* Choose a training set for this tree and use the rest of the test cases to estimate the error.
* For each node of the tree, randomly choose $m$ variables on which to base the decision. Calculate the best partition of the training set from the $m$ variables.


**Advantages Random Forest:**
* runtimes are quite fast
* Are able to deal with unbalanced and missing data



**Random Forest is a prediction ensemble, in which:
* The training dataset :  divided into sub datasets.
* Each sub dataset is processed to generate a prediction model.
* Each prediction model is applied to each test data
* A prediction is by using the “maximum voting” method.

**To illustrate the concept, you can see below:
**


![https://miro.medium.com/max/592/1*i0o8mjFfCn-uD79-F1Cqkw.png](https://miro.medium.com/max/592/1*i0o8mjFfCn-uD79-F1Cqkw.png)

In [ ]:
# We define the number of trees in the forest in 100. 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# We define the model
rfcla = RandomForestClassifier(n_estimators=100,random_state=49,n_jobs=-1)

# We train model
rfcla.fit(X_train, Y_train)

# We predict target values
Y_predict5 = rfcla.predict(X_test)

In [ ]:
Y_predict5

In [ ]:
test_acc_rfcla = round(rfcla.fit(X_train,Y_train).score(X_test, Y_test)* 100, 2)
train_acc_rfcla = round(rfcla.fit(X_train, Y_train).score(X_train, Y_train)* 100, 2)

# Measurement
When the classification process was already done. This work evaluated the results using the Confusion Matrix.

# Confusion matrix

Confusion Matrix is commonly used for a summarization of prediction results
on a classification problem.The number of correct and incorrect predictions 
is summarized with counting values and each value broken down for each class. 
Each of them is the key to the confusion matrix. It shows the classification 
model is confused when it makes predictions, at this point in here it gives us 
insight not only into the errors being made by a classifier but also show the 
types of errors that are being made [[3](https://www.geeksforgeeks.org/confusion-matrix-machine-learning/)]. 

In [ ]:
# The confusion matrix
rfcla_cm = confusion_matrix(Y_test, Y_predict5)
f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(rfcla_cm, annot=True, linewidth=0.7, linecolor='black', fmt='g', ax=ax, cmap="Greens")
plt.title('Random Forest Classification Confusion Matrix')
plt.xlabel('Y predict')
plt.ylabel('Y test')
plt.show()

# Result

# Accuracy
is closeness of the measurements to a specific value. Accuracy has two definitions:
1. More commonly, it is a description of systematic errors, a measure of statistical bias; low accuracy causes a difference between a result and a "true" value. ISO calls this trueness.
1. Alternatively, ISO defines[[1](https://en.wikipedia.org/wiki/Accuracy_and_precision)] accuracy as describing a combination of both types of observational error above (random and systematic), so high accuracy requires both high precision and high trueness.

In [ ]:
model1 = pd.DataFrame({
    'Model': ['Random Forest'],
    'Train Score': [train_acc_rfcla],
    'Test Score': [test_acc_rfcla]
})
model1.sort_values(by='Test Score', ascending=False)

# Precision and Recall

Precision is a description of random errors, a measure of statistical variability.
In simpler terms, given a set of data points from repeated measurements of the same quantity, the set can be said to be accurate if their average is close to the true value of the quantity being measured, while the set can be said to be precise if the values are close to each other. While Recall is defined as the fraction of relevant documents retrieved compared to the total number of relevant documents (true positives divided by true positives+false negatives).

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(Y_test, Y_predict5)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

disp = plot_precision_recall_curve(rfcla,X_train, Y_train)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))